In [ ]:
!wget http://schema.org/docs/tree.jsonld
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark
from pyspark.sql.functions import input_file_name
import time
import re
from pyspark.sql.functions import col
from pyspark.sql import functions as f
#spark = SparkSession.builder.appName("WDC-DataFrame").getOrCreate()
#créer une session dans le master
spark = SparkSession.builder.master("spark://172.20.53.96:7077").appName("WDC-classCount").getOrCreate()
#spark = SparkSession.builder.master("local").appName("WDC-classCount").getOrCreate()


#fichiers de config qui permettent de se connecter au serveur de stockage s3 qui contient les fichiers de DataCommons
endpoint_url = 'https://s3.os-bird.glicid.fr/'
aws_access_key_id = '***REMOVED***'
aws_secret_access_key = '***REMOVED***'
hadoopConf = spark._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', aws_access_key_id)
hadoopConf.set('fs.s3a.secret.key', aws_secret_access_key)
hadoopConf.set('fs.s3a.endpoint', endpoint_url)
hadoopConf.set('fs.s3a.path.style.access', 'true')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

hadoopConf.set('spark.worker.cleanup.enabled', 'true')
hadoopConf.set('fs.s3a.committer.name', 'magic')

In [ ]:
import re
def cleanData(url):
  print(url)
  m = re.search('<https?://schema.org/(.*)>',url)
  if m:
    return m.group(1)
  return ''

In [ ]:
import sys
def class_count_dict(classname):
    result = spark.sql(f"SELECT count FROM CLASSCOUNT WHERE predicate like 'isa:<schema.org/{classname}>'")
    if(result.count() > 0):
        return result.collect()[0]["count"]
    else:
        return 0

In [ ]:
def add_values_to_dict(current_branch):
    #print(child["name"])
    #if current_branch["name"] in class_count_dict:
      current_branch["value"] = class_count_dict(current_branch["name"])
    #else:
    #  current_branch["value"] =0


In [ ]:
def handle_nesting(child):
  #print( "children"   in child)
  if  "children"   in child:
  
    for c in child["children"]:
      add_values_to_dict(c)
      if "children" in c :
        handle_nesting(c)
  

In [ ]:
readall = spark.read.option("header",True) \
  .csv("s3a://test-out/wdcfix/**")

readall.take(5)

import pyspark.sql.functions as f
csall=readall.groupby("pset").agg(f.sum("count").alias('count')).sort(f.desc("count"))

csall.createOrReplaceTempView("CSET")
csall.show(truncate=100)

In [ ]:
from pyspark.sql.functions import col, split
def findISA(topn = None):
    if(topn is not None):
        param = f"limit {topn}"
    else:
        param = ""
    
    sets = csall.withColumn("pset", split(csall["pset"], " "))
    
    distinct_predicate = sets.selectExpr("explode(pset) as predicate", "count") \
                    .groupBy("predicate").agg(f.sum(f.col("count")).alias("count"))  \
                    .createOrReplaceTempView("newsets")
    distinct_predicate=spark.sql(f"select * from newsets where predicate like '%isa:%' and count > 1 order by count desc {param}")    
    return distinct_predicate

In [ ]:
classCount = findISA()
classCount.show(truncate=150)
classCount.createOrReplaceTempView("CLASSCOUNT")

In [ ]:
import json

# Opening JSON file
f = open('tree.jsonld')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

for child in data["children"]:
  handle_nesting(child)

for child in data["children"]:
  add_values_to_dict(child)

In [ ]:
print(data)

In [ ]:
json_object = json.dumps(data, indent=4)
with open("count.json", "w") as outfile:
  outfile.write(json_object)